# **Brief description of the problem and data**

In this competition, our task is to build a Generative Adverserial Network (GAN) model to generate 7,000 to 10,000 Monet-style images.  A GAN consists of at least two neural networks: a generator model and a discriminator model. The generator model is a neural network that creates the images and then trained using the discriminator model. The two models work against each other, with the generator trying to trick the discriminator, while the discriminator trys to accurately classify the real vs. generated images.

The dataset contains two directories:
* monet_tfrec - 300 Monet paintings sized 256x256 in TFRecord format
* photo_tfrec - 7028 photos sized 256x256 in TFRecord format

Alternate directories provided by the compeition contained images in JPEG format yet I choose to work wtih the TFRecords data format. The competition recommended Amy Jang's notebook that goes over the basics of loading data from TFRecords, using TPUs, and building a CycleGAN.

# **Load Libraries**

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa

from kaggle_datasets import KaggleDatasets
import matplotlib.pyplot as plt
import numpy as np

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Device:', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()
print('Number of replicas:', strategy.num_replicas_in_sync)

AUTOTUNE = tf.data.experimental.AUTOTUNE
    
print(tf.__version__)

Number of replicas: 1
2.11.0


# **Exploratory Data Analysis (EDA) — Inspect, Visualize and Clean the Data**

In [2]:
GCS_PATH = KaggleDatasets().get_gcs_path()

BackendError: Unexpected response from the service. Response: {'errors': ['Notebook has no attached datasets. Please attach a dataset before calling get_gcs_path'], 'error': {'code': 9, 'details': []}, 'wasSuccessful': False}.

In [ ]:
MONET_FILENAMES = tf.io.gfile.glob(str(GCS_PATH + '/monet_tfrec/*.tfrec'))
print('Monet TFRecord Files:', len(MONET_FILENAMES))

PHOTO_FILENAMES = tf.io.gfile.glob(str(GCS_PATH + '/photo_tfrec/*.tfrec'))
print('Photo TFRecord Files:', len(PHOTO_FILENAMES))

# **Model Architecture**

# **Results and Analysis**

# **Submission**

# **Conclusion**